<a href="https://colab.research.google.com/github/luanapetrolli/luanapetrolli/blob/main/Salvando_e_Carregando_o_Modelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importanto Pandas

In [ ]:
import pandas as pd

In [ ]:
#carregando o dataset
df = pd.read_csv("/content/drive/MyDrive/projeto-ia-datasets/house-prices-train.csv")

#Mapeamento das features

In [ ]:
cat_vars = df.select_dtypes(include="object").columns.to_list()

target = "SalePrice"

num_vars = [col for col in df.columns if col not in cat_vars+ ["Id",target]]

features = num_vars+ cat_vars

#Treino e Teste

In [ ]:
from sklearn.model_selection import train_test_split

X = df[features]
y= df[target]

X_train, X_teste, y_train, y_teste = train_test_split(X,y, train_size=0.8 , random_state=42)

#Importando o Modelo e a Feature Engine

In [ ]:
!pip install feature-engine lightgbm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.9/328.9 kB 5.6 MB/s eta 0:00:00


In [ ]:
from feature_engine.imputation import ArbitraryNumberImputer, CategoricalImputer
from feature_engine.encoding import OneHotEncoder
from sklearn.pipeline import Pipeline

from lightgbm import LGBMRegressor

#Importando as Métricas

In [ ]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import max_error
from sklearn.metrics import median_absolute_error
from  sklearn.metrics import mean_absolute_percentage_error

#Montando o Pipeline e Treinando

In [ ]:
data_pipe_linear_LGBMRegressor =Pipeline(steps=[
    ("numeric_imputer", ArbitraryNumberImputer(variables=num_vars, arbitrary_number=-999)),
    ("categoric_imputer",CategoricalImputer(fill_value="Missing",variables=cat_vars)),
    ("ohe",OneHotEncoder(variables=cat_vars)),
    ("LGBMRegressor",LGBMRegressor(random_state=42))
])

#treiando
data_pipe_linear_LGBMRegressor.fit(X_train,y_train)

#prevendo
y_previsto= data_pipe_linear_LGBMRegressor.predict(X_teste)

#resultados
print("--------------------------------------")
print("LGBMRegressor")
r2_noteste = r2_score(y_teste, y_previsto)
print(f"R2 no traino {r2_noteste}")
mae_noteste = mean_absolute_error(y_teste, y_previsto)
print(f"MAE no teste {mae_noteste}")
mse_noteste = mean_squared_error(y_teste, y_previsto)
print(f"MSE no teste {mse_noteste}")
rmse_noteste = mean_squared_error(y_teste, y_previsto, squared=False)
print(f"RMSE no teste {rmse_noteste}")
max_error_noteste = max_error(y_teste, y_previsto)
print(f"max_erro no teste {max_error_noteste}")
medae_error_noteste = median_absolute_error(y_teste, y_previsto)
print(f"medae no teste {medae_error_noteste}")
mape_error_noteste = mean_absolute_percentage_error(y_teste, y_previsto)
print(f"mape no teste {mape_error_noteste}")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001107 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3221
[LightGBM] [Info] Number of data points in the train set: 1168, number of used features: 193
[LightGBM] [Info] Start training from score 181441.541952
--------------------------------------
LGBMRegressor
R2 no traino 0.8893698106000011
MAE no teste 16752.873857558716
MSE no teste 848569641.0608867
RMSE no teste 29130.218692294206
max_erro no teste 234554.32028490375
medae no teste 9622.275100271538
mape no teste 0.10005989678712546


#Salvando o modelo utilizando Pickle e Joblib

leitura complemetar: https://acervolima.com/salvando-um-modelo-de-aprendizado-de-maquina/

No aprendizado de máquina, ao trabalhar com a biblioteca scikit learn , precisamos salvar os modelos treinados em um arquivo e restaurá-los para reutilizá-lo para comparar o modelo com outros modelos, para testar o modelo em um novo dado. O salvamento dos dados é chamado de serialização , enquanto a restauração dos dados é chamada de desserialização .
Além disso, lidamos com diferentes tipos e tamanhos de dados. Alguns conjuntos de dados são facilmente treinados, ou seja, levam menos tempo para treinar, mas os conjuntos de dados cujo tamanho é grande (mais de 1 GB) podem levar muito tempo para treinar em uma máquina local, mesmo com GPU. Quando precisarmos dos mesmos dados treinados em algum projeto diferente ou posteriormente em algum momento, para evitar o desperdício de tempo de treinamento, armazene o modelo treinado para que possa ser usado a qualquer momento no futuro.
Existem duas maneiras de salvar um modelo no scikit learn: Joblib e Pickle.

#Pickle String
o módulo pickle implementa um algoritmo fundamental, mas poderoso, para serializar e desserializar uma estrutura de objeto Python.
     
O modelo Pickle fornece as seguintes funções:

- pickle.dump para serializar uma hierarquia de objetos, você simplesmente usa dump().

- pickle.load para desserializar um fluxo de dados, você chama a função carrega().

In [ ]:
#importando a biblioteca
import pickle

#Salvar o modelo treinado como uma string em formato pickle.
saved_model = pickle.dumps(data_pipe_linear_LGBMRegressor)

# Defina o caminho do arquivo onde você deseja salvar o modelo.
model_path = "/content/drive/MyDrive/model.pkl"

# Escreva o modelo serializado no local de arquivo especificado.
with open(model_path, "wb") as file:
    file.write(saved_model)

In [ ]:
# Define o caminho do arquivo contendo o modelo
model_path = "/content/drive/MyDrive/model.pkl"

# Carrega o modelo a partir do arquivo
with open(model_path, "rb") as file:
    loaded_model = pickle.load(file)

# Use o modelo carregado em formato pickle para fazer previsões.
loaded_model.predict(X_teste)

array([145485.60717921, 312602.055066  , 110022.00247526, 159969.41493332,
       313880.38905875,  79552.66858545, 236131.91780394, 149779.28043869,
        77235.80685494, 135095.91132809, 143800.35902572, 121835.25269376,
       115362.62424022, 195750.94518798, 178634.86874112, 129730.62507722,
       192911.56084663, 131657.45654703, 120820.37087536, 204017.77278542,
       138338.9114947 , 225369.63892082, 171268.62434023, 129775.13104949,
       196638.99285044, 171349.93326745, 199272.04783511,  97943.32721604,
       175722.41999571, 208295.25799675, 130129.51288706, 249081.95544949,
       192519.84994714, 118052.59576667, 259595.70902645, 150151.35384487,
       118264.44369995, 209968.20236273, 333180.50781153, 106279.34445848,
       129130.50704429, 236915.99730642, 121245.73799971, 396585.63553583,
       127844.04225973, 126381.94538548, 117616.87247532, 122589.06581806,
       408127.7881369 , 142933.05040505, 119082.89270624, 191035.39925391,
       120004.75943429, 3

#Modelo em conserva como um arquivo usando joblib
Joblib é a substituição do pickle, pois é mais eficiente em objetos que carregam arrayes grandes numpy. Essas funções também aceitam objetos semelhantes a arquivos em vez de nomes de arquivos.

- joblib.dump para serializar uma hierarquia de objetos

- joblib.load para desserializar um fluxo de dados

In [ ]:
#importando a biblioteca
import joblib

#Salvar o modelo como um objeto pickle em um arquivo.
joblib.dump(data_pipe_linear_LGBMRegressor, '/content/drive/MyDrive/filename20231021.pkl')

# Carregando o modelo de um arquivo
knn_from_joblib = joblib.load('/content/drive/MyDrive/filename20231021.pkl')

# Utilizando o modelo para fazer predições
knn_from_joblib.predict(X_teste)

array([145485.60717921, 312602.055066  , 110022.00247526, 159969.41493332,
       313880.38905875,  79552.66858545, 236131.91780394, 149779.28043869,
        77235.80685494, 135095.91132809, 143800.35902572, 121835.25269376,
       115362.62424022, 195750.94518798, 178634.86874112, 129730.62507722,
       192911.56084663, 131657.45654703, 120820.37087536, 204017.77278542,
       138338.9114947 , 225369.63892082, 171268.62434023, 129775.13104949,
       196638.99285044, 171349.93326745, 199272.04783511,  97943.32721604,
       175722.41999571, 208295.25799675, 130129.51288706, 249081.95544949,
       192519.84994714, 118052.59576667, 259595.70902645, 150151.35384487,
       118264.44369995, 209968.20236273, 333180.50781153, 106279.34445848,
       129130.50704429, 236915.99730642, 121245.73799971, 396585.63553583,
       127844.04225973, 126381.94538548, 117616.87247532, 122589.06581806,
       408127.7881369 , 142933.05040505, 119082.89270624, 191035.39925391,
       120004.75943429, 3